pip install pandas numpy scipy statsmodels matplotlib gspread oauth2client

Integrated data smoothing steps
Fixed OWID-Wiki test data interaction - now takes exclusively OWID data from first available data, but will accept Wiki data before that

**NOTE:** If getting errors during or at end of smoothing process, double check renames and droplist against updated OWID test data
**NOTE 2:** If .csv files are showing up blank, check google sheet and make sure there are enough columns!

In [1]:
import requests
import json
import gspread
import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from shutil import copy
from scipy import interpolate
from statsmodels.tsa.seasonal import STL
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
def download_sheets(sheet_key, testsheet_url, cred_json, scope, datalist, 
                    testsheet_renames=None, testsheet_droplist=None, retain_wikidata=None):
    """ Downloads Google spreadsheet and saves each tab as CSV """
    credentials = ServiceAccountCredentials.from_json_keyfile_name(cred_json, scope)
    gc = gspread.authorize(credentials)
    book = gc.open_by_key(sheet_key)
    print("Downloading data from Google sheets...")
    
    def update_testdata(testdata, testsheet_url, testsheet_renames=None, 
                        testsheet_droplist=None, retain_wikidata=None):
        """ Supplements testing numbers using data from OurWorldInData 
        and COVID tracking project (for US only) """
        # Download & read in test data from OurWorldInData spreadsheet
        print("Downloading OWID test data...")
        testdf = pd.read_csv(testsheet_url)

        # Set up OWID dataframe with index and column labels
        testdf = testdf.filter(['Entity','Date','Cumulative total'], axis=1)
        print(testdf)
        testdf.Entity.astype(str)
        
        # Drop country-repeat data
        if testsheet_droplist:
            for d in testsheet_droplist:
                testdf = testdf[testdf.Entity != d]    
        
        testdf.Entity = testdf.Entity.str.split(" - ",expand=True)[0]
        testdf.Date = ((pd.to_datetime(testdf.Date) - pd.datetime(*[2019,10,15])).dt.days * 1.0)
        testdf = testdf[testdf.Date > 94]
        testdf['Cumulative total'] = pd.to_numeric(testdf['Cumulative total'], errors='coerce')
        
        testdf = pd.pivot_table(testdf, index='Entity', columns='Date')
        testdf.drop(['United States'], inplace=True) # Drop USA data (will get from covidtracking.com)
        if testsheet_renames:
            testdf.rename(index=testsheet_renames, inplace=True)    

        # Set up testdata dataframe with index and column labels
        header = testdata.iloc[0]
        header[2:] = header[2:].astype(int)
        testdata = testdata[1:]
        testdata.columns = header        
        testdata.index = testdata.iloc[:,1]
        print(testdf)

        # Write any OWID data over testdata values starting from first available date of OWID data
        print("Updating OWID test data...")
        for rownum, row in testdf.iterrows():
            if rownum in retain_wikidata:
                row.loc[:get_first_idx(row)] = -1 # Identify pre-first date data with placeholder
                for idx, val in row.items():
                    if testdf.at[rownum, idx] != -1:
                        testdata.at[rownum, idx[1]] = val
            else:
                for idx, val in row.items():
                    testdata.at[rownum, idx[1]] = val

        # Download US test data from covidtracking.com
        print("Updating US test data...")
        usdaily_json = requests.get('https://covidtracking.com/api/us/daily').json()
        usdaily = pd.DataFrame(usdaily_json)
        ustotal = usdaily.filter(['date','totalTestResults'], axis=1)

        ustotal.date = ((pd.to_datetime(ustotal.date, format='%Y%m%d') - pd.datetime(*[2019,10,15])).dt.days * 1.0)

        ustotal.index = ustotal.date
        usdata = ustotal.drop('date', axis=1).T
        usdata.rename(index={'totalTestResults':'USA'}, inplace=True)

        for col, series in usdata.items():
            for row, val in series.items():
                testdata.at[row, col] = val
        
        limit = (pd.to_datetime("today") - pd.datetime(*[2019,10,15])).days - 95
        testdata = testdata.columns.to_frame().T.append(testdata, ignore_index=True)
        testdata = testdata.iloc[:, :limit]
        return testdata
    
    for dataname in datalist:
        worksheet = book.worksheet(dataname)
        table = worksheet.get_all_values()
        dataframe = pd.DataFrame(table)
        
        # For column 2 in ConstantData, strip % signs and convert to float
        if dataname == "ConstantData":
            dataframe[3][1:] = pd.to_numeric(dataframe[3][1:].str.strip('%')).div(100)
        
        # For row 191 (USA) in TestData, replce with CovidTracker data
        if dataname == "TestData":
            dataframe = update_testdata(dataframe, testsheet_url, testsheet_renames, 
                                        testsheet_droplist, retain_wikidata)
            
        dataframe.to_csv(f'{dataname}.csv', header=False, index=False)

        
def import_datasets(datalist, vdfname):
    """ Creates Vensim script to convert CSVs to VDFXs """
    print("Importing data to VDF...")
    scenario_text = []
    scenario_text.append("SPECIAL>NOINTERACTION\n")
    
    for dataname in datalist:
        scenario_text.append(f"MENU>CSV2VDF|{dataname}.csv|{vdfname}{dataname}.vdfx|{dataname}.frm|\n")
    
    scenario_text.append("MENU>EXIT\n")
    
    scriptfile = open("ImportData.cmd", 'w')
    scriptfile.writelines(scenario_text)
    scriptfile.close()

    
def copy_data(datalist, vdfname):
    """ Copies VDFXs to parent directory of working directory """
    for dataname in datalist:
        for filetype in [".vdf", ".vdfx"]:
            try:
                copy(f"./{vdfname}{dataname}{filetype}", f"../")
            except FileNotFoundError:
                pass

            
def idx_to_int(df):
    """Converts string numeric column keys of dataframe to int"""
    Tdf = df.T
    Tdf.index = Tdf.index.astype('float').astype('int')
    newdf = Tdf.T
    return(newdf)


def get_first_idx(s):
    return (s > 0).idxmax(skipna=True)


def get_last_idx(s):
    return s.where(s > 0).last_valid_index()


def calculate_devs(flowrow, windowlength, datathreshold, thresholdwidth=1):
    """Calculate rolling mean of series and adjusted deviations from the mean, as well as 
    threshold values based on median +/- MADs, ignoring values below given datathreshold"""
    flowmeanraw = flowrow.rolling(windowlength, min_periods=1, center=True).mean()
    flowmean = flowmeanraw.copy()
    flowmean.loc[:(flowmean >= datathreshold).idxmax()] = np.nan
    flowrawdev = flowrow - flowmean
    flowadjdev = flowrawdev / np.sqrt(flowmean)
    lowthreshold = flowadjdev.median() - flowadjdev.mad() * thresholdwidth
    highthreshold = flowadjdev.median() + flowadjdev.mad() * thresholdwidth
    devs = {'rawmean': flowmeanraw, 'mean': flowmean, 'rawdev': flowrawdev, 
            'adjdev': flowadjdev, 'lowthr': lowthreshold, 'highthr': highthreshold}
    return devs


def fill_dips(smflow, smdevs, k, smoothfactor, lowthreshold, borrowlength=7):
    """Identify points with deviations below threshold value and partially fill 
    by borrowing from following points, based on a multinomial draw with probabilities 
    proportional to deviations of those points"""
    for i, adjdev in enumerate(smdevs['adjdev'][:-k]):
        if adjdev < lowthreshold:
            borrowlist = smdevs['adjdev'].iloc[i+1:max(i+1+borrowlength, i+1)]
            values = smflow.iloc[i+1:max(i+1+borrowlength, i+1)]
            borrowlist -= adjdev
            borrowlist.mask(borrowlist < 0, other=0, inplace=True)
            if not all([(b == 0 or np.isnan(b)) for b in borrowlist]):
                borrowlist.astype('float64')
                borrowlist.dropna(inplace=True)
                borrowlist /= borrowlist.sum()
                mnlist = np.random.multinomial(abs(int(np.floor(smdevs['rawdev'].iloc[i]*smoothfactor))), 
                                               [abs(i) for i in borrowlist])
                minlen = min(len(mnlist), len(values))
                mnlist = np.minimum(mnlist[:minlen], values[:minlen])
                smflow.iloc[i] += mnlist.sum()
                for j, val in enumerate(mnlist):
                    smflow.iloc[i+1+j] -= val

                
def smooth_peaks(smflow, smdevs, k, smoothfactor, highthreshold, distlength=14):
    """Identify points with deviations above threshold value and partially flatten 
    by distributing to preceding points, based on a multinomial draw with probabilities 
    proportional to existing rolling means of those points"""
    for i, adjdev in reversed(list(enumerate(smdevs['adjdev'][:-k]))):
        if adjdev > highthreshold:
            distlist = smdevs['rawmean'].iloc[max(0, i-distlength):i]
            if not all([(d == 0 or np.isnan(d)) for d in distlist]):
                distlist.astype('float64')
                distlist /= distlist.sum()
                mnlist = np.random.multinomial(abs(int(np.floor(smdevs['rawdev'].iloc[i]*smoothfactor))), distlist)
                smflow.iloc[i] -= mnlist.sum()
                for j, val in enumerate(mnlist):
                    smflow.iloc[i-len(mnlist)+j] += val


def iter_smooth(smflow, ordevs, windowlength, datathreshold, smoothfactor, 
                borrowlength=7, distlength=14, iterlimit=10):
    """Iteratively apply dip-filling and peak-smoothing algorithms until 
    all deviations are within the upper and lower median+/-MAD thresholds"""
    smdevs = calculate_devs(smflow, windowlength, datathreshold)
    i = 0
    while i < iterlimit:
        # If mean values are too low, skip all smoothing
        if np.nanmax(smdevs['mean']) < datathreshold:
            break
        # Identify last valid index and check if below threshold
        k = smflow.index.get_loc(get_last_idx(smflow))
        k = len(smflow) - k
        # Identify all consecutive final terms below threshold to skip, otherwise will cause errors
        while smdevs['adjdev'].iloc[-k] < ordevs['lowthr']:
            k +=1
        if np.nanmin(smdevs['adjdev'][:-k]) < ordevs['lowthr']:
            fill_dips(smflow, smdevs, k, smoothfactor, ordevs['lowthr'])
            smdevs = calculate_devs(smflow, windowlength, datathreshold)
        if np.nanmax(smdevs['adjdev'][:-k]) > ordevs['highthr']:
            smooth_peaks(smflow, smdevs, k, smoothfactor, ordevs['highthr'])
            smdevs = calculate_devs(smflow, windowlength, datathreshold)
        if (np.nanmax(smdevs['adjdev'][:-k]) < ordevs['highthr'] 
            and np.nanmin(smdevs['adjdev'][:-k]) > ordevs['lowthr']):
            break
        i += 1
    return smflow


def cross_corr(x, y, shift):
    """Get time-shifted cross-correlations of two series"""
    if shift > 0:
        xshift = x[0:-shift]
        yshift = y[shift:]
    elif shift < 0:
        xshift = x[-shift:]
        yshift = y[0:shift]
    elif shift == 0:
        xshift = x
        yshift = y

    rawcorrs = np.correlate(xshift, yshift, mode='full')
    normcorr = rawcorrs[(rawcorrs.size // 2):] / np.amax(rawcorrs)
    
    return normcorr[0]


def time_shift(x, shift):
    """Shift a series by a specified amount"""
    xshift = x.copy()
    if shift > 0:
        xshift[shift:] = x[0:-shift]
    elif shift < 0:
        xshift[0:shift] = x[-shift:]
    elif shift == 0:
        pass
    return xshift

    
def smooth_data(skiplist):
    """Run data smoothing and time shifting on data"""    
    print("Executing smoothing algorithm!")
        
    # Import dataframes from CSV and drop variable names
    testdf = pd.read_csv("TestData.csv", index_col=1,header=0)
    testdf.drop(columns='Time', inplace=True)

    formdf = pd.read_csv("FormattedData.csv", index_col=1,header=0)
    formdf.drop(columns='Time', inplace=True)

    flowdf = pd.read_csv("FlowData.csv",index_col=1,header=0)
    flowdf.drop(columns='Time', inplace=True)

    # Convert string indices to int
    testdf = idx_to_int(testdf)
    formdf = idx_to_int(formdf)
    flowdf = idx_to_int(flowdf)
    
    display(testdf)

    # Set up sub-dataframes from main data files
    infdf = flowdf[0:nrows].copy()
    dthdf = flowdf[nrows:(nrows*2)].copy()
    recdf = flowdf[(nrows*2):(nrows*3)].copy()
#     tratedf = pd.DataFrame(data=None, columns=testdf.columns, index=testdf.index)
#     tcapdf = pd.DataFrame(data=None, columns=testdf.columns, index=testdf.index)
#     tratedf = testdf.replace(testdf, np.nan)
#     tcapdf = testdf.replace(testdf, np.nan)
    tratedf = pd.DataFrame().reindex_like(testdf)
    tcapdf = pd.DataFrame().reindex_like(testdf)
    
    # Convert infinite values to NaN to avoid potential errors
    testdf.replace([np.inf, -np.inf], np.NaN)
    
    for i in testdf.index:
        # Check if country is in skiplist
        if i in skiplist:
            print(f"Repressing {i}!")
            continue
        
        # Check if country has sufficient test data to proceed, else skip
        elif len(testdf.loc[i].dropna()) > mintestpoints:

            # Ensure cumulative test data is strictly monotonic increasing
            # NOTE: if monotonicity check happens after date value assignment, 
            # then if last test data point is nonmonotonic, it will be dropped causing an error
            testdf.loc[i] = testdf.loc[i].mask(testdf.loc[i].cummax().duplicated())

            # Identify first and last infection, test, and death date indices
            infA, testA = [get_first_idx(s) for s in [infdf.loc[i], testdf.loc[i]]]
            infZ, testZ, dthZ = [get_last_idx(s) for s in [infdf.loc[i], testdf.loc[i], dthdf.loc[i]]]

            # Assign 0 test value to first infection date if before first test date
            if infA < testA:
                newtestA = infA
                testdf.loc[i, newtestA] = 0
            else:
                newtestA = testA

            # Set test rate and capacity values to 0 before first data point
            tratedf.loc[i, :newtestA], tcapdf.loc[i, :newtestA] = 0, 0

            # Check whether original test data is sparse in latter half of test data window
            halftestrow = testdf.loc[i, newtestA:testZ]
            halftestrow = halftestrow.iloc[len(halftestrow)//2:]
            if len(halftestrow.dropna())/len(halftestrow) > 0.5:
                smcheck = False
            else:
                smcheck = True
                print(i, "is sparse:", len(testdf.loc[i]), len(halftestrow), len(halftestrow.dropna()))

            # Interpolate test data using PCHIP spline if possible, within range of presumed test data
            spline = interpolate.PchipInterpolator(testdf.loc[i].dropna().index, testdf.loc[i].dropna().values)
            interptests = spline(testdf.loc[i, newtestA:testZ].index)

            # Check if any interpolated values are negative; if so do linear interpolation instead
            if any((interptests[1:] - interptests[:-1]) < 0):
                print("Uh-oh, negative spline result, going linear!")
                linear = interpolate.interp1d(testdf.loc[i].dropna().index, testdf.loc[i].dropna().values)
                interptests = linear(testdf.loc[i, newtestA:testZ].index)

            # Assign interpolated values back to test data
            testdf.loc[i, newtestA:testZ] = interptests
            tratedf.loc[i, newtestA:testZ] = np.insert((interptests[1:] - interptests[:-1]), 0, interptests[0])

            # If original test data is sparse, smooth test and infection data
            if smcheck:
                tratedevs = calculate_devs(tratedf.loc[i, newtestA:testZ], windowlength, datathreshold)
                tratedf.loc[i, newtestA:testZ] = iter_smooth(tratedf.loc[i, newtestA:testZ], tratedevs, 
                                                             windowlength, datathreshold, smoothfactor)
                infdevs = calculate_devs(infdf.loc[i, :infZ], windowlength, datathreshold)
                infdf.loc[i, :infZ] = iter_smooth(infdf.loc[i, :infZ], infdevs, windowlength, datathreshold, smoothfactor)

            # Else if original test data not sparse, do time shift on test data
            else:
                minlen = min(len(tratedf.loc[i].dropna()), len(infdf.loc[i].dropna()))
                if minlen == 0:
                    print(f"Insufficient data for {i} shift, skipping!")
                else:
                    x = STL(tratedf.loc[i].dropna(), period=7, seasonal=7).fit().seasonal
                    y = STL(infdf.loc[i].dropna(), period=7, seasonal=7).fit().seasonal

                    alseas = x.align(y, join='inner')

                    seascorrs = []
                    shiftrange = list(range(-2,5))

                    for j in shiftrange:
                        seascorrs.append(cross_corr(alseas[0], alseas[1], j))

                    tshift = shiftrange[np.argmax(seascorrs)]
                    shifttrate = time_shift(tratedf.loc[i], tshift)

                    tratedf.loc[i] = shifttrate
                    newtestA += tshift
                    testZ += tshift

                    print(f"{i} shift is {tshift}")

            # Run polyfit on test rate data for later use to estimate test capacity
            # Test capacity will be estimated as max of fitted test rate LATER on whole DF
            pfit = np.polyfit(tratedf.loc[i, newtestA:testZ].index, tratedf.loc[i, newtestA:testZ].values, 10)
            tcapdf.loc[i, newtestA:testZ] = np.polyval(pfit, tratedf.loc[i, newtestA:testZ].index)

            # Run iterative dip/peak smoothing on death rates for all countries with enough deaths
            if np.nanmax(dthdf.loc[i]) > datathreshold:
                dthdevs = calculate_devs(dthdf.loc[i, :dthZ], windowlength, datathreshold)
                dthdf.loc[i, :dthZ] = iter_smooth(dthdf.loc[i, :dthZ], dthdevs, windowlength, datathreshold, smoothfactor)
                
        else:
            print(f"Not enough test data for {i}, skipping!")
    
    # Combine flow data streams into one dataframe
    smflowdf = pd.concat([infdf, dthdf, recdf], axis=0)

    # Set test capacity based on polyfit of test rate, ignoring first day
    tcapdf.iloc[:, 1:] = tcapdf.iloc[:, 1:].cummax(axis=1, skipna=False)

    # Recalculate cumulative tests based on smoothed test data
    testdf = tratedf.cumsum(axis=1, skipna=False)

    # Combine all three test data streams into one dataframe, dropping first day
    smtestdf = pd.concat([testdf, tratedf, tcapdf], axis=0).iloc[:,1:]

    # Shave NANs and last column of test dataframe
    smtestdf.dropna(axis=1, how='all', inplace=True)
    smtestdf = smtestdf.iloc[:,:-1]

    # Adjust first day flows to account for non-zero initial cumulative values
    smflowdf.iloc[:,0] += formdf.iloc[:,0]

    # Recalculate cumulative data from smoothed flows, then readjust first day flows
    smformdf = smflowdf.cumsum(axis=1)
    smflowdf.iloc[:,0] -= formdf.iloc[:,0]

    # Restore variable names and export to CSV
    smflowdf.reset_index(inplace=True)
    smflowdf.insert(0, 'Time', ['DataFlowInfection']*nrows+['DataFlowDeath']*nrows+['DataFlowRecovery']*nrows)
    smflowdf.to_csv("FlowData.csv", index=False)

    smtestdf.reset_index(inplace=True)
    smtestdf.insert(0, 'Time', ['DataCmltTest']*nrows+['DataTestRate']*nrows+['DataTestCapacity']*nrows)
    smtestdf.to_csv("TestData.csv", index=False)

    smformdf.reset_index(inplace=True)
    smformdf.insert(0, 'Time', ['DataCmltInfection']*nrows+['DataCmltDeath']*nrows+['DataCmltRecovery']*nrows)
    smformdf.to_csv("FormattedData.csv", index=False)
        


In [3]:
controlfilename = input("Enter control file name (with extension):")
controlfile = json.load(open(controlfilename, 'r'))

# Unpack controlfile into variables
for k,v in controlfile.items():
    exec(k + '=v')

download_sheets(sheet_key, testsheet_url, cred_json, scope, datalist,
                testsheet_renames, testsheet_droplist, retain_wikidata)

Enter control file name (with extension):V7DUControl.txt
                           Entity        Date  Cumulative total
0       Argentina - people tested  2020-01-01               1.0
1       Argentina - people tested  2020-02-03               2.0
2       Argentina - people tested  2020-02-04               3.0
3       Argentina - people tested  2020-02-05               NaN
4       Argentina - people tested  2020-02-06               NaN
...                           ...         ...               ...
31620  Zimbabwe - tests performed  2020-12-17          193011.0
31621  Zimbabwe - tests performed  2020-12-18          195166.0
31622  Zimbabwe - tests performed  2020-12-19          196747.0
31623  Zimbabwe - tests performed  2020-12-20          198891.0
31624  Zimbabwe - tests performed  2020-12-21          200096.0

[31625 rows x 3 columns]


C:\Users\tylim\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
C:\Users\tylim\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\tylim\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

           Cumulative total                                                  \
Date                  95.0  96.0  97.0  98.0  99.0  100.0 101.0 102.0 103.0   
Entity                                                                        
Argentina               NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
Australia               NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
Austria                 NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
Bahrain                 NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
Bangladesh              NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
...                     ...   ...   ...   ...   ...   ...   ...   ...   ...   
UK                      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
Uruguay                 NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
Vietnam                 NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
Zambia                  NaN   NaN   NaN   NaN   NaN 

C:\Users\tylim\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:63: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
C:\Users\tylim\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\tylim\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [4]:
if smoothing == True:
    for k,v in smparams.items():
        exec(k + '=v')
    smooth_data(skiplist)

import_datasets(datalist, vdfname)

Executing smoothing algorithm!


,95,96,97,98,99,100,101,102,103,104,...,423,424,425,426,427,428,429,430,431,432
Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venezuela,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Vietnam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1469955.0,NaN
Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zambia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,471542.0,477174.0,484299.0,489926.0,493685.0,498639.0,505896.0,513928.0,518986.0,524876.0


Not enough test data for Afghanistan, skipping!
Albania is sparse: 338 47 9
Not enough test data for Algeria, skipping!
Not enough test data for Andorra, skipping!
Not enough test data for Angola, skipping!
Not enough test data for Antigua, skipping!
Argentina shift is 0
Armenia is sparse: 338 43 7
Australia shift is -1
Austria shift is 0
Azerbaijan is sparse: 338 46 8
Not enough test data for Bahamas, skipping!
Bahrain shift is 2
Bangladesh shift is 0
Not enough test data for Barbados, skipping!
Belarus is sparse: 338 135 47
Belgium shift is 1
Not enough test data for Belize, skipping!
Not enough test data for Benin, skipping!
Not enough test data for Bhutan, skipping!
Bolivia shift is 0
Bosnia shift is -1
Not enough test data for Botswana, skipping!
Repressing Brazil!
Not enough test data for Brunei, skipping!
Bulgaria shift is -1
Not enough test data for Burkina Faso, skipping!
Not enough test data for Burundi, skipping!
Not enough test data for CaboVerde, skipping!
Not enough test 

C:\Users\tylim\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:233: RuntimeWarning: invalid value encountered in true_divide


IvoryCoast shift is -2
Jamaica shift is 0
Japan shift is 1
Jordan is sparse: 338 147 50
Kazakhstan shift is -2
Kenya shift is 0
Not enough test data for Kiribati, skipping!
Not enough test data for Kosovo, skipping!
Kuwait shift is 0
Kyrgyzstan is sparse: 338 25 6


...


Netherlands is sparse: 338 149 22
Not enough test data for NewGuinea, skipping!
NewZealand shift is 1
Not enough test data for Nicaragua, skipping!
Not enough test data for Niger, skipping!
Nigeria shift is 2
Not enough test data for NorthKorea, skipping!
NorthMacedonia shift is 1
Norway shift is 1
Not enough test data for Oman, skipping!
Pakistan shift is -1
Not enough test data for Palau, skipping!
Panama shift is 0
Paraguay shift is 0
Peru shift is 2
Philippines shift is 2
Poland shift is 0
Portugal shift is 1
Qatar shift is 0
Romania shift is 0
Russia shift is 2
Rwanda shift is 4
Not enough test data for SaintKitts, skipping!
Not enough test data for SaintLucia, skipping!
Not enough test data for Samoa, skipping!
Not enough test data for San Marino, skipping!
Not enough test data for SaoTome, skipping!
SaudiArabia shift is 1
Senegal shift is 0
Serbia shift is 0
Not enough test data for Seychelles, skipping!
Not enough test data for Sierra Leone, skipping!
Singapore is sparse: 338 

C:\Users\tylim\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:233: RuntimeWarning: invalid value encountered in true_divide


Turkey shift is -1
Not enough test data for Turkmenistan, skipping!
Not enough test data for Tuvalu, skipping!
UAE shift is 0
Uganda shift is 1
UK shift is 0
Ukraine shift is 0
Uruguay shift is 0
USA shift is 0
Not enough test data for Uzbekistan, skipping!
Not enough test data for Vanuatu, skipping!
Not enough test data for Vatican, skipping!
Not enough test data for Venezuela, skipping!
Vietnam is sparse: 338 166 36
Not enough test data for Yemen, skipping!
Zambia shift is 0


C:\Users\tylim\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:233: RuntimeWarning: invalid value encountered in true_divide


Zimbabwe shift is -2
Importing data to VDF...


In [5]:
subprocess.run(f"{vensimpath} \"./ImportData.cmd\"", check=True)

copy_data(datalist, vdfname)
print("Job done!")

Job done!


In [ ]:
testdf = pd.read_csv("TestData.csv", index_col=1,header=0)
testdf.drop(columns='Time', inplace=True)
display(testdf)

newdf = pd.DataFrame().reindex_like(testdf)
display(newdf)